In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as p
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.spatial import distance
from scipy.stats.mstats import gmean
%matplotlib inline
from itertools import combinations
from itertools import chain
import sys
import os
import copy
sns.set_style('white')
sns.set_style('ticks')
sns.set_color_codes()

fgm_simulation_path = '/Users/grantkinsler/Documents/Stanford/Research/StarryNight/Git/starry-night/Simulations/FGM_simulation_callable.py'
sys.path.append(os.path.dirname(os.path.expanduser(fgm_simulation_path)))
from FGM_simulation_callable import simulation, nball_pull, gaussian_fitness

tools_path = '../code/tools.py'
sys.path.append(os.path.dirname(os.path.expanduser(tools_path)))
import tools

In [3]:
np.random.seed(953527608) # for exact figure reproducibility use this seed

In [4]:
fitness_data = p.read_csv('../data/DoubleBC_Merged_Fitness_Atish_Default_AllConditions.csv')
fitness_data = p.read_csv('../data/DoubleBC_Merged_Fitness_Atish_Weighted_Default_AllConditions_IncludingOld_swapsremoved.csv')

In [5]:
fitness_data = fitness_data.replace([np.inf, -np.inf], np.nan).dropna()

In [6]:
fitness_data

,barcode,gene,type,ploidy,class,additional_muts,13.1_error,13.1_fitness,13.2_error,13.2_fitness,...,1BB_1%Raf_fitness,1BB_1%Raf_error,1BB_0.5%Raf_fitness,1BB_0.5%Raf_error,1BB_1%Gly_fitness,1BB_1%Gly_error,1BB_1%EtOH_fitness,1BB_1%EtOH_error,1BB_SucRaf_fitness,1BB_SucRaf_error
0,53,Diploid,Diploid,Diploid,Diploid,TIP1-upstream_point_variant; YKR012C-upstream_...,0.086877,0.366047,0.083404,0.315150,...,0.097501,0.048235,0.267181,0.024068,0.347187,0.030065,0.537630,0.021632,0.381319,0.065192
1,151,IRA1,stop_gained,Haploid,PKA,SEH1-missense_variant; ZIP1-missense_variant; ...,0.059241,1.628226,0.059251,1.570717,...,1.039126,0.042740,0.994910,0.016550,1.038026,0.023238,0.715633,0.016767,0.595644,0.050561
2,262,NotSequenced,NotSequenced,NotSequenced,NotSequenced,NotSequenced,0.069475,0.196277,0.068634,0.105044,...,0.019456,0.052719,0.286933,0.025276,0.345255,0.031072,0.493757,0.023397,0.350548,0.067009
3,273,IRA1,frameshift_variant,Haploid,PKA,None,0.059303,1.133521,0.059304,1.206520,...,0.846375,0.042806,0.775972,0.016898,0.812373,0.023670,0.494898,0.018183,0.452001,0.051965
4,323,NotSequenced,NotSequenced,NotSequenced,NotSequenced,NotSequenced,0.059382,1.113742,0.059383,1.101149,...,0.671028,0.043155,0.628388,0.017825,0.663332,0.024495,0.433916,0.019959,0.291819,0.055298
8,689,IRA1,frameshift_variant,Haploid,PKA,RPL19A-upstream_point_variant,0.059639,1.351715,0.059628,1.287908,...,1.143336,0.042876,0.986495,0.017254,1.089611,0.024627,0.688321,0.022166,0.504585,0.054950
9,697,NotSequenced,NotSequenced,NotSequenced,NotSequenced,NotSequenced,0.079211,0.292709,0.078981,0.195564,...,0.137316,0.050013,0.215339,0.025871,0.299675,0.032186,0.490181,0.023398,0.369475,0.069865
12,1379,NotSequenced,NotSequenced,NotSequenced,NotSequenced,NotSequenced,0.059238,1.474213,0.059241,1.421350,...,0.980880,0.042745,0.864478,0.016595,0.983229,0.023286,0.446068,0.017521,0.416034,0.050919
13,1488,NotSequenced,NotSequenced,NotSequenced,NotSequenced,NotSequenced,0.059386,1.346518,0.059399,1.306377,...,1.092418,0.042772,0.944628,0.016728,1.035003,0.023628,0.429861,0.019687,0.561952,0.051527
14,1564,Diploid,Diploid,Diploid,Diploid,None,0.083986,0.379311,0.087251,0.141528,...,-0.000640,0.052389,0.178112,0.026440,0.315670,0.033512,0.506919,0.024411,0.346503,0.072595


In [7]:
gene_type_combos = np.unique([(g,t) for g,t in zip(fitness_data['gene'].values,fitness_data['type'].values)],axis=0)

In [8]:
bc_list = []

for (g,t) in gene_type_combos:
    if not (('other' in g) or ('NotSequenced' in g)):
        this_gt = fitness_data[(fitness_data['gene'].isin([g]) & fitness_data['type'].isin([t]))]
        n_samples = int(np.floor(len(this_gt.index)/2))
        print(g,t,len(this_gt.index),n_samples)
        
        bc_list = bc_list + list(np.random.choice(this_gt['barcode'].values,n_samples))

exp_neutral =  fitness_data[fitness_data['class'].isin(['ExpNeutral'])]
n_samples = int(np.floor(len(exp_neutral.index)/2))
bc_list = bc_list + list(np.random.choice(exp_neutral['barcode'].values,n_samples)) 


fitness_data[fitness_data['barcode'].isin(bc_list)].to_csv('../data/mutant_train_set.csv',index=False)
## should this be balanced by mutation type (diploids dominate...)          

CYR1 missense_variant 3 1
Diploid Diploid 169 84
Diploid + Chr11Amp Diploid + Chr11Amp 3 1
Diploid + IRA1 missense_variant 1 0
Diploid + IRA2 frameshift_variant 1 0
Diploid + IRA2 missense_variant 1 0
Diploid + IRA2 stop_gained 1 0
GPB1 frameshift_variant 1 0
GPB1 missense_variant 1 0
GPB1 stop_gained 1 0
GPB2 frameshift_variant 5 2
GPB2 missense_variant 1 0
GPB2 stop_gained 8 4
IRA1 frameshift_variant 7 3
IRA1 missense_variant 9 4
IRA1 stop_gained 9 4
IRA2 missense_variant 7 3
KOG1 missense_variant 1 0
PDE2 frameshift_variant 6 3
PDE2 missense_variant 2 1
PDE2 stop_gained 3 1
RAS2 missense_variant 1 0
SCH9 missense_variant 1 0
TFS1 missense_variant 1 0
TOR1 missense_variant 1 0


In [9]:
typical_test_list = []

number_per = 10

for (g,t) in gene_type_combos:
    if not (('other' in g) or ('NotSequenced' in g)):
        this_gt = fitness_data[(fitness_data['gene'].isin([g]) & fitness_data['type'].isin([t]))]
        n_samples = min([int(np.floor(len(this_gt.index))),number_per])
        print(g,t,len(this_gt.index),n_samples)
        
        typical_test_list = typical_test_list + list(np.random.choice(this_gt['barcode'].values,n_samples))

exp_neutral =  fitness_data[fitness_data['class'].isin(['ExpNeutral'])]
n_samples =  min([int(np.floor(len(this_gt.index))),number_per])
typical_test_list = typical_test_list + list(np.random.choice(exp_neutral['barcode'].values,n_samples)) 

typical_test_list = typical_test_list + list(fitness_data[fitness_data['gene']=='other']['barcode'].values) + list(fitness_data[fitness_data['gene']=='NotSequenced']['barcode'].values)

typical_test_list = [bc for bc in typical_test_list if bc not in bc_list]

fitness_data[fitness_data['barcode'].isin(typical_test_list)].to_csv('../data/mutant_test_set.csv',index=False)        

CYR1 missense_variant 3 3
Diploid Diploid 169 10
Diploid + Chr11Amp Diploid + Chr11Amp 3 3
Diploid + IRA1 missense_variant 1 1
Diploid + IRA2 frameshift_variant 1 1
Diploid + IRA2 missense_variant 1 1
Diploid + IRA2 stop_gained 1 1
GPB1 frameshift_variant 1 1
GPB1 missense_variant 1 1
GPB1 stop_gained 1 1
GPB2 frameshift_variant 5 5
GPB2 missense_variant 1 1
GPB2 stop_gained 8 8
IRA1 frameshift_variant 7 7
IRA1 missense_variant 9 9
IRA1 stop_gained 9 9
IRA2 missense_variant 7 7
KOG1 missense_variant 1 1
PDE2 frameshift_variant 6 6
PDE2 missense_variant 2 2
PDE2 stop_gained 3 3
RAS2 missense_variant 1 1
SCH9 missense_variant 1 1
TFS1 missense_variant 1 1
TOR1 missense_variant 1 1


In [10]:
fitness_data[fitness_data['barcode'].isin(bc_list)].to_csv('../data/mutant_train_set.csv',index=False)

In [11]:
fitness_data[fitness_data['barcode'].isin(bc_list)]

,barcode,gene,type,ploidy,class,additional_muts,13.1_error,13.1_fitness,13.2_error,13.2_fitness,...,1BB_1%Raf_fitness,1BB_1%Raf_error,1BB_0.5%Raf_fitness,1BB_0.5%Raf_error,1BB_1%Gly_fitness,1BB_1%Gly_error,1BB_1%EtOH_fitness,1BB_1%EtOH_error,1BB_SucRaf_fitness,1BB_SucRaf_error
0,53,Diploid,Diploid,Diploid,Diploid,TIP1-upstream_point_variant; YKR012C-upstream_...,0.086877,0.366047,0.083404,0.315150,...,0.097501,0.048235,0.267181,0.024068,0.347187,0.030065,0.537630,0.021632,0.381319,0.065192
8,689,IRA1,frameshift_variant,Haploid,PKA,RPL19A-upstream_point_variant,0.059639,1.351715,0.059628,1.287908,...,1.143336,0.042876,0.986495,0.017254,1.089611,0.024627,0.688321,0.022166,0.504585,0.054950
15,1617,PDE2,frameshift_variant,Haploid,PKA,RIF1-upstream_indel_variant,0.060159,0.958329,0.060073,1.097166,...,0.794548,0.042834,0.841260,0.016921,0.770074,0.023680,0.410900,0.018495,0.479477,0.051868
19,2037,Diploid,Diploid,Diploid,Diploid,None,0.076847,0.360570,0.076824,0.305632,...,0.080084,0.046161,0.291905,0.021712,0.379325,0.027089,0.539239,0.019584,0.400161,0.058922
22,2468,IRA2,missense_variant,Haploid,PKA,SEC5-missense_variant,0.063501,0.916300,0.062726,0.865927,...,0.895482,0.043049,0.798387,0.017983,0.750201,0.025619,0.302979,0.026209,0.412359,0.054813
26,2808,Diploid,Diploid,Diploid,Diploid,CDC39-missense_variant,0.069829,0.359467,0.067649,0.271368,...,0.407979,0.046292,0.397011,0.021252,0.470161,0.026571,0.518873,0.021013,0.412059,0.063089
30,3379,Diploid,Diploid,Diploid,Diploid,DTR1-stop_gained,0.070338,0.339854,0.068991,0.246891,...,0.038059,0.046104,0.294850,0.020922,0.347093,0.026392,0.484319,0.019053,0.351486,0.057150
37,3920,Diploid,Diploid,Diploid,Diploid,None,0.078640,0.224272,0.077516,0.106723,...,0.114790,0.060249,0.202851,0.031110,0.246377,0.043619,0.499260,0.031965,0.414712,0.091274
40,4791,Diploid,Diploid,Diploid,Diploid,tL(CAA)G3-upstream_point_variant,0.065383,0.633945,0.064357,0.596045,...,0.432133,0.045518,0.458777,0.020839,0.522857,0.026587,0.758183,0.019494,0.404874,0.062321
41,4949,IRA1,missense_variant,Haploid,PKA,None,0.059602,1.027491,0.059592,0.976031,...,0.905912,0.042843,0.646613,0.017066,0.802816,0.023628,0.518548,0.018530,0.303473,0.052532


In [12]:
exp_neutral

,barcode,gene,type,ploidy,class,additional_muts,13.1_error,13.1_fitness,13.2_error,13.2_fitness,...,1BB_1%Raf_fitness,1BB_1%Raf_error,1BB_0.5%Raf_fitness,1BB_0.5%Raf_error,1BB_1%Gly_fitness,1BB_1%Gly_error,1BB_1%EtOH_fitness,1BB_1%EtOH_error,1BB_SucRaf_fitness,1BB_SucRaf_error
292,72939,NotSequenced,NotSequenced,NotSequenced,ExpNeutral,NotSequenced,0.094346,0.035435,0.095718,0.020351,...,0.0268,0.054952,0.003748,0.031132,-0.021662,0.04797,0.042239,0.040955,0.006673,0.086473


In [13]:
minimal_bc_list = []

number_per = 2

for (g,t) in gene_type_combos:
    if not (('other' in g) or ('NotSequenced' in g)):
        this_gt = fitness_data[(fitness_data['gene'].isin([g]) & fitness_data['type'].isin([t]))]
        n_samples = min([int(np.floor(len(this_gt.index)/2)),number_per])
        print(g,t,len(this_gt.index),n_samples)
        
        minimal_bc_list = minimal_bc_list + list(np.random.choice(this_gt['barcode'].values,n_samples))

exp_neutral =  fitness_data[fitness_data['class'].isin(['ExpNeutral'])]
n_samples = int(np.floor(len(exp_neutral.index)/2))
minimal_bc_list = minimal_bc_list + list(np.random.choice(exp_neutral['barcode'].values,n_samples))  
## should this be balanced by mutation type (diploids dominate...)          

CYR1 missense_variant 3 1
Diploid Diploid 169 2
Diploid + Chr11Amp Diploid + Chr11Amp 3 1
Diploid + IRA1 missense_variant 1 0
Diploid + IRA2 frameshift_variant 1 0
Diploid + IRA2 missense_variant 1 0
Diploid + IRA2 stop_gained 1 0
GPB1 frameshift_variant 1 0
GPB1 missense_variant 1 0
GPB1 stop_gained 1 0
GPB2 frameshift_variant 5 2
GPB2 missense_variant 1 0
GPB2 stop_gained 8 2
IRA1 frameshift_variant 7 2
IRA1 missense_variant 9 2
IRA1 stop_gained 9 2
IRA2 missense_variant 7 2
KOG1 missense_variant 1 0
PDE2 frameshift_variant 6 2
PDE2 missense_variant 2 1
PDE2 stop_gained 3 1
RAS2 missense_variant 1 0
SCH9 missense_variant 1 0
TFS1 missense_variant 1 0
TOR1 missense_variant 1 0


In [14]:
fitness_data[fitness_data['barcode'].isin(minimal_bc_list)].to_csv('../data/mutant_minimal_train_set.csv',index=False)

In [15]:
minimal_test_list = []

number_per = 10

for (g,t) in gene_type_combos:
    if not (('other' in g) or ('NotSequenced' in g)):
        this_gt = fitness_data[(fitness_data['gene'].isin([g]) & fitness_data['type'].isin([t]))]
        n_samples = min([int(np.floor(len(this_gt.index))),number_per])
        print(g,t,len(this_gt.index),n_samples)
        
        minimal_test_list = minimal_test_list + list(np.random.choice(this_gt['barcode'].values,n_samples))

exp_neutral =  fitness_data[fitness_data['class'].isin(['ExpNeutral'])]
n_samples =  min([int(np.floor(len(this_gt.index))),number_per])
minimal_test_list = minimal_test_list + list(np.random.choice(exp_neutral['barcode'].values,n_samples)) 

minimal_test_list = minimal_test_list + list(fitness_data[fitness_data['gene']=='other']['barcode'].values) + list(fitness_data[fitness_data['gene']=='NotSequenced']['barcode'].values)

minimal_test_list = [bc for bc in minimal_test_list if bc not in minimal_bc_list]

fitness_data[fitness_data['barcode'].isin(minimal_test_list)].to_csv('../data/mutant_minimal_test_set.csv',index=False)        

CYR1 missense_variant 3 3
Diploid Diploid 169 10
Diploid + Chr11Amp Diploid + Chr11Amp 3 3
Diploid + IRA1 missense_variant 1 1
Diploid + IRA2 frameshift_variant 1 1
Diploid + IRA2 missense_variant 1 1
Diploid + IRA2 stop_gained 1 1
GPB1 frameshift_variant 1 1
GPB1 missense_variant 1 1
GPB1 stop_gained 1 1
GPB2 frameshift_variant 5 5
GPB2 missense_variant 1 1
GPB2 stop_gained 8 8
IRA1 frameshift_variant 7 7
IRA1 missense_variant 9 9
IRA1 stop_gained 9 9
IRA2 missense_variant 7 7
KOG1 missense_variant 1 1
PDE2 frameshift_variant 6 6
PDE2 missense_variant 2 2
PDE2 stop_gained 3 3
RAS2 missense_variant 1 1
SCH9 missense_variant 1 1
TFS1 missense_variant 1 1
TOR1 missense_variant 1 1
